In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import glob
%matplotlib inline

# Define the file pattern to match all CSV files
file_pattern = 'dispense_report_*.csv'

# Use glob to get all file paths matching the pattern
file_paths = glob.glob(file_pattern)

# Read and concatenate all CSV files into one DataFrame
data_frames = [pd.read_csv(file) for file in file_paths]
data = pd.concat(data_frames, ignore_index=True)

# Extract relevant features for clustering
X = data[['OD Sphere', 'OD Cylinder', 'OD Axis', 'OD Add',
          'OS Sphere', 'OS Cylinder', 'OS Axis', 'OS Add']]

# Handle missing values (if any)
X.fillna(0, inplace=True)

# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [53]:
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
import matplotlib.pyplot as plt
import numpy as np

# Calculate Z-scores
z_scores = np.abs((X_scaled - X_scaled.mean(axis=0)) / X_scaled.std(axis=0))

# Filter out rows where any Z-score is above the threshold (e.g., 3)
threshold = 3
data = data[(z_scores < threshold).all(axis=1)]

X = data[['OD Sphere', 'OD Cylinder', 'OD Axis', 'OD Add',
          'OS Sphere', 'OS Cylinder', 'OS Axis', 'OS Add']]
# Standardize the filtered data
X_scaled = scaler.fit_transform(X)

# Perform hierarchical clustering on filtered data
Z = linkage(X_scaled, method='ward')



In [ ]:

# Plot the dendrogram
plt.figure(figsize=(10, 7))
dendrogram(Z)
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Glasses')
plt.ylabel('Distance')
plt.show()

# Cut the dendrogram to form clusters (e.g., 5 clusters)
num_clusters = 6
data['cluster'] = fcluster(Z, t=num_clusters, criterion='maxclust')

# Add cluster labels to the original data
data['cluster'] = data['cluster'].astype(str)  # Convert to string for easier analysis

In [ ]:

from sklearn.decomposition import PCA
# Apply PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Explained variance ratio
print("Explained variance ratio:", pca.explained_variance_ratio_)

# Loading scores
loading_scores = pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2'], index=X.columns)
print("Loading scores:\n", loading_scores)

# Plot the PCA components
plt.figure(figsize=(8, 6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.5)
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('PCA of Dispense Report Data')
plt.show()

# Plot loading scores
fig, ax = plt.subplots(1, 2, figsize=(14, 6))
loading_scores['PC1'].plot(kind='bar', ax=ax[0])
ax[0].set_title('Loading Scores for PC1')
ax[0].set_ylabel('Loading Score')
ax[0].set_xlabel('Feature')
ax[0].set_ylim(-0.5, 0.5)

loading_scores['PC2'].plot(kind='bar', ax=ax[1])
ax[1].set_title('Loading Scores for PC2')

plt.tight_layout()
plt.show()

In [ ]:
# Apply PCA
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

# Explained variance ratio
print("Explained variance ratio:", pca.explained_variance_ratio_)

# Loading scores
loading_scores = pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2', 'PC3'], index=X.columns)
print("Loading scores:\n", loading_scores)

# Plot the PCA components in 3D
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2], c=data['cluster'].astype(int), cmap='viridis', alpha=0.5)

ax.set_xlabel('PCA Component 1')
ax.set_ylabel('PCA Component 2')
ax.set_zlabel('PCA Component 3')
ax.set_title('3D PCA of Dispense Report Data')
legend1 = ax.legend(*scatter.legend_elements(), title="Clusters")

plt.show()

# Plot loading scores
fig, ax = plt.subplots(1, 3, figsize=(18, 6))
loading_scores['PC1'].plot(kind='bar', ax=ax[0])
ax[0].set_title('Loading Scores for PC1')
ax[0].set_ylabel('Loading Score')
ax[0].set_xlabel('Feature')
ax[0].set_ylim(-0.5, 0.5)

loading_scores['PC2'].plot(kind='bar', ax=ax[1])
ax[1].set_title('Loading Scores for PC2')
ax[1].set_ylabel('Loading Score')
ax[1].set_xlabel('Feature')
ax[1].set_ylim(-0.5, 0.5)

loading_scores['PC3'].plot(kind='bar', ax=ax[2])
ax[2].set_title('Loading Scores for PC3')
ax[2].set_ylabel('Loading Score')
ax[2].set_xlabel('Feature')
ax[2].set_ylim(-0.5, 0.5)

plt.tight_layout()
plt.show()

In [ ]:
# Load inventory data
inventory_data = pd.read_csv('inventory_sa.csv')

# Extract relevant features for clustering from inventory data
inventory_features = inventory_data[['OD Sphere', 'OD Cylinder', 'OD Axis', 'OD Add',
                                     'OS Sphere', 'OS Cylinder', 'OS Axis', 'OS Add']]

# Handle missing values (if any)
inventory_features.fillna(0, inplace=True)


In [ ]:
# Calculate dispense frequency for each type of glasses from dispense data
dispense_frequency = data.groupby(['OD Sphere', 'OD Cylinder', 'OD Axis', 'OD Add',
                                   'OS Sphere', 'OS Cylinder', 'OS Axis', 'OS Add']).size().reset_index(name='dispense_frequency')

# Merge dispense frequency with inventory data
merged_data = pd.merge(dispense_frequency, inventory_data, on=['OD Sphere', 'OD Cylinder', 'OD Axis', 'OD Add',
                                                               'OS Sphere', 'OS Cylinder', 'OS Axis', 'OS Add'], how='right')

# Fill NaN values in dispense_frequency with 0 (for glasses that were never dispensed)
merged_data['dispense_frequency'].fillna(0, inplace=True)

In [ ]:
# Analyze dispense and inventory by cluster
cluster_summary = merged_data.groupby('cluster').agg({
    'dispense_frequency': 'sum',  # Total dispenses in the cluster
    'SKU': 'count'               # Total inventory in the cluster (count of SKUs)
}).reset_index()

print("Cluster Summary:")
print(cluster_summary)

# Identify clusters with high dispense frequency and low inventory
high_demand_clusters = cluster_summary[(cluster_summary['dispense_frequency'] > cluster_summary['dispense_frequency'].quantile(0.75)) & 
                                       (cluster_summary['SKU'] < cluster_summary['SKU'].quantile(0.25))]

print("High-demand, low-inventory clusters:")
print(high_demand_clusters)

In [ ]:
# Filter glasses in high-demand, low-inventory clusters
high_demand_glasses = merged_data[merged_data['cluster'].isin(high_demand_clusters['cluster'])]

# Sort by dispense frequency to prioritize restocking
high_demand_glasses = high_demand_glasses.sort_values(by='dispense_frequency', ascending=False)

print("Glasses to restock:")
print(high_demand_glasses[['OD Sphere', 'OD Cylinder', 'OD Axis', 'OD Add',
                           'OS Sphere', 'OS Cylinder', 'OS Axis', 'OS Add',
                           'dispense_frequency', 'SKU', 'cluster']])